![](https://koyomi.oss-cn-hangzhou.aliyuncs.com/20200209134347.png)

# 数据预处理

In [147]:
import pandas as pd
import numpy as np

In [148]:
train_file = "./5-1.csv"
train_data = pd.read_csv(train_file)

In [149]:
print(train_data)

    ID  年龄 有工作 有自己的房子 信贷情况 类别
0    1  青年   否      否   一般  否
1    2  青年   否      否    好  否
2    3  青年   是      否    好  是
3    4  青年   是      是   一般  是
4    5  青年   否      否   一般  否
5    6  中年   否      否   一般  否
6    7  中年   否      否    好  否
7    8  中年   是      是    好  是
8    9  中年   否      是  非常好  是
9   10  中年   否      是  非常好  是
10  11  老年   否      是  非常好  是
11  12  老年   否      是    好  是
12  13  老年   是      否    好  是
13  14  老年   是      否  非常好  是
14  15  老年   否      否   一般  否


In [150]:
type(train_data)

pandas.core.frame.DataFrame

In [151]:
train_data.head()

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,青年,否,否,一般,否
1,2,青年,否,否,好,否
2,3,青年,是,否,好,是
3,4,青年,是,是,一般,是
4,5,青年,否,否,一般,否


In [152]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
ID        15 non-null int64
年龄        15 non-null object
有工作       15 non-null object
有自己的房子    15 non-null object
信贷情况      15 non-null object
类别        15 non-null object
dtypes: int64(1), object(5)
memory usage: 800.0+ bytes


In [153]:
age_data = train_data['年龄'].copy()
age_data

0     青年
1     青年
2     青年
3     青年
4     青年
5     中年
6     中年
7     中年
8     中年
9     中年
10    老年
11    老年
12    老年
13    老年
14    老年
Name: 年龄, dtype: object

In [154]:
for i,x in enumerate(age_data):
    if x == '青年':
        age_data[i] = 1
    elif x == '中年':
        age_data[i] = 2
    elif x == '老年':
        age_data[i] = 3

train_data['年龄'] = age_data

In [155]:
train_data

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,1,否,否,一般,否
1,2,1,否,否,好,否
2,3,1,是,否,好,是
3,4,1,是,是,一般,是
4,5,1,否,否,一般,否
5,6,2,否,否,一般,否
6,7,2,否,否,好,否
7,8,2,是,是,好,是
8,9,2,否,是,非常好,是
9,10,2,否,是,非常好,是


In [156]:
work_data = train_data['有工作'].copy()
for i,x in enumerate(work_data):
    if x == '是':
        work_data[i] = 1
    if x == '否':
        work_data[i] = 2
train_data['有工作'] = work_data

In [157]:
train_data

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,1,2,否,一般,否
1,2,1,2,否,好,否
2,3,1,1,否,好,是
3,4,1,1,是,一般,是
4,5,1,2,否,一般,否
5,6,2,2,否,一般,否
6,7,2,2,否,好,否
7,8,2,1,是,好,是
8,9,2,2,是,非常好,是
9,10,2,2,是,非常好,是


In [158]:
house_data = train_data['有自己的房子'].copy()
for i,x in enumerate(house_data):
    if x == '是':
        house_data[i] = 1
    if x == '否':
        house_data[i] = 2
train_data['有自己的房子'] = house_data

In [159]:
loan_data = train_data['信贷情况'].copy()
for i,x in enumerate(loan_data):
    if x == '非常好':
        loan_data[i] = 1
    if x == '好':
        loan_data[i] = 2
    if x == '一般':
        loan_data[i] = 3
train_data['信贷情况'] = loan_data

In [160]:
y_data = train_data['类别'].copy()
for i,x in enumerate(y_data):
    if x == '是':
        y_data[i] = 1
    if x == '否':
        y_data[i] = 2
train_data['类别'] = y_data

In [161]:
train_data

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,1,2,2,3,2
1,2,1,2,2,2,2
2,3,1,1,2,2,1
3,4,1,1,1,3,1
4,5,1,2,2,3,2
5,6,2,2,2,3,2
6,7,2,2,2,2,2
7,8,2,1,1,2,1
8,9,2,2,1,1,1
9,10,2,2,1,1,1


# C4.5
## 计算经验商
$H(D)=-\sum_{k=1}^{K}\frac{|C_k|}{D}log_2\frac{C_k}{D}$

In [162]:
import math
K = 2
D = train_data.shape[0]
C = np.zeros([K])
C
D

15

In [163]:
def H_func(D,n):
    d = D.shape[0]
    c = np.zeros(n)
    c = [D.loc[D['类别']==(i+1)].shape[0]/d for i,x in enumerate(c)]
    print(c)
    return np.sum([[-x*math.log2(x) if x!=0 else 0 for x in c]])

In [164]:
H_D = H_func(train_data,K)

[0.6, 0.4]


## 计算经验条件商
$H(D|A)=\sum_{i=1}^n\frac{|D_i|}{|D|}H(D_i)$

In [165]:
# D: 求解集合 A: 求解条件 n:条件的取值种类
def H_A_func(D,A,n):
    H = np.zeros(n)
    H = [D.loc[D[A]==(i+1)].shape[0]/D.shape[0]*H_func(D.loc[D[A]==(i+1)],K) for i,x in enumerate(H)]
    return np.sum(H)

In [166]:
H_A_func(train_data,'年龄',3)

[0.4, 0.6]
[0.6, 0.4]
[0.8, 0.2]


0.8879430945988998

In [167]:
H_A_func(train_data,'有自己的房子',2)

[1.0, 0.0]
[0.3333333333333333, 0.6666666666666666]


0.5509775004326937

In [168]:
H_A_func(train_data,'有工作',2)

[1.0, 0.0]
[0.4, 0.6]


0.6473003963031123

In [169]:
H_A_func(train_data,'信贷情况',3)

[1.0, 0.0]
[0.6666666666666666, 0.3333333333333333]
[0.2, 0.8]


0.6079610319175832

In [170]:
H_A=[H_A_func(train_data,'年龄',3),H_A_func(train_data,'有工作',2),
     H_A_func(train_data,'有自己的房子',2),H_A_func(train_data,'信贷情况',3)]

[0.4, 0.6]
[0.6, 0.4]
[0.8, 0.2]
[1.0, 0.0]
[0.4, 0.6]
[1.0, 0.0]
[0.3333333333333333, 0.6666666666666666]
[1.0, 0.0]
[0.6666666666666666, 0.3333333333333333]
[0.2, 0.8]


In [171]:
H_A

[0.8879430945988998,
 0.6473003963031123,
 0.5509775004326937,
 0.6079610319175832]

## 计算信息增益

In [172]:
g = [H_D - x for x in H_A]
g

[0.08300749985576883,
 0.32365019815155627,
 0.4199730940219749,
 0.36298956253708536]

## 计算信息增益比

In [173]:
gR = [x/H_D for x in g]

In [174]:
gR

[0.08549096146585063,
 0.3333333333333334,
 0.4325380677663126,
 0.3738496733100589]

## 递归构造树
信息增益比最大是A3 *有自己的房子*

A3 有两个可能值:1,2 将其分割为两个非空子集

In [182]:
D_A3_1 = train_data.loc[train_data['有自己的房子']==1]
D_A3_1

,ID,年龄,有工作,有自己的房子,信贷情况,类别
3,4,1,1,1,3,1
7,8,2,1,1,2,1
8,9,2,2,1,1,1
9,10,2,2,1,1,1
10,11,3,2,1,1,1
11,12,3,2,1,2,1


In [183]:
D_A3_2 = train_data.loc[train_data['有自己的房子']==2]
D_A3_2

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,1,2,2,3,2
1,2,1,2,2,2,2
2,3,1,1,2,2,1
4,5,1,2,2,3,2
5,6,2,2,2,3,2
6,7,2,2,2,2,2
12,13,3,1,2,2,1
13,14,3,1,2,1,1
14,15,3,2,2,3,2


In [184]:
H_D_A3_1 = H_func(D_A3_1,K)
H_D_A3_1

[1.0, 0.0]


0.0

In [185]:
if H_D_A3_1 != 0:
    H_A3_1 = [H_A_func(D_1,'年龄',3)/H_D_A3_1,H_A_func(D_1,'有工作',2)/H_D_A3_1,
      H_A_func(D_1,'信贷情况',3)/H_D_A3_1]
else :
    print('D_1')


D_1


In [187]:
H_D_A3_2 = H_func(D_A3_2,K)
H_A3_2 = [(H_D_A3_2-H_A_func(D_2,'年龄',3))/H_D_A3_2,(H_D_A3_2-H_A_func(D_2,'有工作',2))/H_D_A3_2,
      (H_D_A3_2-H_A_func(D_2,'信贷情况',3))/H_D_A3_2]
H_A3_2

[0.3333333333333333, 0.6666666666666666]
[0.25, 0.75]
[0.0, 1.0]
[0.6666666666666666, 0.3333333333333333]
[1.0, 0.0]
[0.0, 1.0]
[1.0, 0.0]
[0.5, 0.5]
[0.0, 1.0]


[0.274017542121281, 1.0, 0.5160116947475206]

![](https://koyomi.oss-cn-hangzhou.aliyuncs.com/20200212115138.png)
          

# 平方误差准则生成回归树

In [188]:
train_data_2 = np.asarray([4.5,4.75,4.91,5.34,
                           5.80,7.05,7.9,8.23,8.7,9])

In [189]:
train_data_2

array([4.5 , 4.75, 4.91, 5.34, 5.8 , 7.05, 7.9 , 8.23, 8.7 , 9.  ])

## 遍历选择最优(j,s)

In [248]:
def cal_c1_c2(R1,R2):
#     print(R1)
#     print(R2)
#     print('\n')
    c1 = 0 if R1.shape[0] == 0 else np.sum(R1)/R1.shape[0]
    c2 = 0 if R2.shape[0] == 0 else np.sum(R2)/R2.shape[0]
    return np.sum([(x-c1)*(x-c1) for x in R1])+np.sum([(x-c2)*(x-c2) for x in R2])

In [249]:
j_s_value = np.zeros(train_data_2.shape[0])

In [250]:
j_s_value = [cal_c1_c2(train_data_2[0:i+1],
                       R2 = train_data_2[i+1:]) 
             for i,x in enumerate(train_data_2)]
j_s_value

[22.648,
 17.702237500000003,
 12.193485714285714,
 7.3787,
 3.358719999999999,
 5.073958333333332,
 10.052466666666668,
 15.177800000000003,
 21.328,
 27.63236]

In [251]:
min_j = np.where(j_s_value == np.min(j_s_value))[0][0]
min_j_s = (min_j,train_data_2[min_j])
min_j_s

(4, 5.8)

这样就求出符合要求的js是(4, 5.8)
## 函数化求解过程

In [252]:
def cal_min_j_s(dataset):
    j_s_value = np.zeros(dataset.shape[0])
    j_s_value = [cal_c1_c2(dataset[0:i+1],
                       R2 = dataset[i+1:]) 
             for i,x in enumerate(dataset)]
    min_j = np.where(j_s_value == np.min(j_s_value))[0][0]
    return (min_j,train_data_2[min_j])

In [253]:
cal_min_j_s(train_data_2)

(4, 5.8)

## 递归求解

In [254]:
def generate_tree(dataset):
    (j,s) = cal_min_j_s(dataset)
    print((j,s))
    dataset_left = dataset[0:j+1]
    dataset_right = dataset[j+1:]
    if dataset_left.shape[0] > 1:
        generate_tree(dataset_left)
    else:
        print("leaf:"+str(dataset_left[0]))
    if dataset_right.shape[0] > 1:
        generate_tree(dataset_right)
    else:
        print("leaf:"+str(dataset_right[0]))

In [255]:
generate_tree(train_data_2)

(4, 5.8)
(2, 4.91)
(0, 4.5)
leaf:4.5
(0, 4.5)
leaf:4.75
leaf:4.91
(0, 4.5)
leaf:5.34
leaf:5.8
(1, 4.75)
(0, 4.5)
leaf:7.05
leaf:7.9
(0, 4.5)
leaf:8.23
(0, 4.5)
leaf:8.7
leaf:9.0


![](https://koyomi.oss-cn-hangzhou.aliyuncs.com/20200212160614.png)